# Mapillary API Examples

This notebook demonstrates how to use the Mapillary API to fetch street-level imagery data and visualize it on an interactive map using Folium.

## Prerequisites

- Mapillary API token (set as environment variable API_TOKEN or MAPPILLARY_API_TOKEN)
- Required packages: geopandas, folium, requests, etc.

In [1]:
# Import required libraries
import os
import folium
from mapillary_api import *
import geopandas as gpd
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Check if API token is available
token = os.environ.get('API_TOKEN') or os.environ.get('MAPPILLARY_API_TOKEN') or get_mapillary_token()
if not token:
    print("⚠️  No API token found. Please set the API_TOKEN environment variable.")
    print("   For GitHub Actions, this should be configured as a repository secret.")
else:
    print("✅ API token found and ready to use!")
    # Show partial token for verification (first 8 chars)
    print(f"   Token preview: {token[:8]}...")

✅ API token found and ready to use!
   Token preview: MLY|4036...


## Example 1: Simple Area Query

Let's fetch Mapillary images from a specific geographic area (using sample region if available)

In [3]:
# Try to use sample region if available, otherwise use a default location
try:
    # Load sample region from existing file
    sample_region = gpd.read_file('sample_region.geojson')
    bbox = sample_region.geometry[0].bounds
    print(f"Using sample region bounds: {bbox}")
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lon = (bbox[0] + bbox[2]) / 2
except:
    # Fallback to a default location (Curitiba, Brazil area)
    print("Sample region not found, using default location (Curitiba, Brazil)")
    center_lat, center_lon = -25.4284, -49.2733
    # Small area around the center
    buffer = 0.005
    bbox = (center_lon - buffer, center_lat - buffer, center_lon + buffer, center_lat + buffer)

print(f"Center coordinates: ({center_lat}, {center_lon})")
print(f"Bounding box: {bbox}")

Using sample region bounds: (-49.277848575951, -25.44461617379733, -49.27585315300058, -25.442756427067422)
Center coordinates: (-25.443686300432375, -49.27685086447579)
Bounding box: (-49.277848575951, -25.44461617379733, -49.27585315300058, -25.442756427067422)


In [4]:
# Fetch Mapillary image metadata for the area
if token:
    print("Fetching Mapillary image metadata...")
    try:
        # Get metadata using the API
        metadata = get_mapillary_images_metadata(
            bbox[0], bbox[1], bbox[2], bbox[3], 
            token=token,
            limit=100  # Limit for demonstration
        )
        
        # Convert to GeoDataFrame
        gdf = mapillary_data_to_gdf(metadata)
        
        print(f"Found {len(gdf)} images in the area")
        if len(gdf) > 0:
            print("First few records:")
            print(gdf[['captured_at', 'compass_angle', 'geometry']].head())
        
    except Exception as e:
        print(f"Error fetching data: {e}")
        # Create empty GeoDataFrame for demonstration
        gdf = gpd.GeoDataFrame()
else:
    print("Skipping data fetch - no API token available")
    gdf = gpd.GeoDataFrame()

Fetching Mapillary image metadata...


Found 100 images in the area
First few records:
     captured_at  compass_angle                     geometry
0  1565959000000           65.4  POINT (-49.27741 -25.44353)
1  1565962472000           69.7  POINT (-49.27623 -25.44293)
2  1565978282000          155.7   POINT (-49.2771 -25.44315)
3  1565962476000           61.0   POINT (-49.27586 -25.4428)
4  1565958995000           62.9  POINT (-49.27778 -25.44367)


## Example 2: Interactive Map Visualization with Folium

Now let's create an interactive map showing the Mapillary image locations.

**Note:** The map uses direct HTML rendering to avoid Jupyter notebook trust issues. The map is also saved as an HTML file for external viewing.

In [5]:
# Create a Folium map centered on our area
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=15,
    tiles='OpenStreetMap'
)

# Add a marker for the center point
folium.Marker(
    [center_lat, center_lon],
    popup='Center Point',
    tooltip='Query Center',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Add bounding box rectangle
bbox_coords = [
    [bbox[1], bbox[0]],  # SW corner
    [bbox[3], bbox[0]],  # NW corner
    [bbox[3], bbox[2]],  # NE corner
    [bbox[1], bbox[2]],  # SE corner
    [bbox[1], bbox[0]]   # Back to SW corner
]

folium.PolyLine(
    bbox_coords,
    color='blue',
    weight=2,
    opacity=0.8,
    popup='Query Bounding Box'
).add_to(m)

# Add Mapillary image locations if we have data
if len(gdf) > 0:
    print(f"Adding {len(gdf)} Mapillary image locations to the map...")
    
    for idx, row in gdf.iterrows():
        if hasattr(row.geometry, 'x') and hasattr(row.geometry, 'y'):
            # Create popup with image information
            popup_text = f"""
            <b>Mapillary Image</b><br>
            <b>ID:</b> {row.get('id', 'N/A')}<br>
            <b>Captured:</b> {row.get('captured_at', 'N/A')}<br>
            <b>Compass Angle:</b> {row.get('compass_angle', 'N/A')}°<br>
            <b>Coordinates:</b> ({row.geometry.y:.5f}, {row.geometry.x:.5f})
            """
            
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                radius=3,
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=f"Mapillary Image {row.get('id', '')[:8]}...",
                color='green',
                fillColor='lightgreen',
                fillOpacity=0.7
            ).add_to(m)
else:
    print("No Mapillary data to display on map")

# Add a title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Mapillary Images Visualization</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Save the map to an HTML file for external viewing
map_filename = 'mapillary_visualization.html'
m.save(map_filename)
print(f"\n📋 Map saved to {map_filename} for external viewing")

# Display the map using direct HTML rendering to avoid iframe trust issues
from IPython.display import HTML
HTML(f'<div style="height: 600px; border: 1px solid #ccc;">{m.get_root().render()}</div>')

Adding 100 Mapillary image locations to the map...

📋 Map saved to mapillary_visualization.html for external viewing


## Example 3: Query by Place Name

Let's try fetching data for a specific place using the territory polygon functionality

In [6]:
# Query a specific place
place_name = "Água Verde, Curitiba, Brazil"
print(f"Trying to get polygon for: {place_name}")

try:
    # Get territory polygon
    polygon_data = get_territory_polygon(place_name)
    
    if polygon_data:
        print("✅ Successfully retrieved territory polygon")
        print(f"Polygon type: {polygon_data['type']}")
        
        # Create a simple map showing the territory
        coords = polygon_data['coordinates'][0]
        
        # Calculate center
        lats = [coord[1] for coord in coords]
        lons = [coord[0] for coord in coords]
        center_lat = sum(lats) / len(lats)
        center_lon = sum(lons) / len(lons)
        
        # Create map
        territory_map = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=13
        )
        
        # Add the polygon
        folium.Polygon(
            locations=[[coord[1], coord[0]] for coord in coords],
            popup=f'Territory: {place_name}',
            tooltip=place_name,
            color='purple',
            fillColor='lightpurple',
            fillOpacity=0.3
        ).add_to(territory_map)
        
        print("Territory map created successfully!")
        
    else:
        print("❌ Could not retrieve territory polygon")
        territory_map = None
        
except Exception as e:
    print(f"Error retrieving territory: {e}")
    territory_map = None

Trying to get polygon for: Água Verde, Curitiba, Brazil


Error retrieving territory: Expecting value: line 1 column 1 (char 0)


In [7]:
# Display the territory map if available
if 'territory_map' in locals() and territory_map is not None:
    territory_map
else:
    print("Territory map not available")

Territory map not available


## Summary

This notebook demonstrates:

1. **API Integration**: How to use the Mapillary API with environment variables for authentication
2. **Data Fetching**: Retrieving street-level imagery metadata for specific geographic areas
3. **Interactive Visualization**: Using Folium to create interactive maps showing:
   - Query bounding boxes
   - Mapillary image locations as clickable markers
   - Territory polygons for named places
4. **Error Handling**: Graceful fallbacks when API tokens or data are unavailable

### Next Steps

- Experiment with different geographic areas
- Modify the query parameters (limit, fields, etc.)
- Try the tiled querying functionality for larger areas
- Download actual images using the `download_all_pictures_from_gdf` function

### GitHub Actions Integration

This notebook is designed to run in GitHub Actions with the `API_TOKEN` secret configured. The workflow will:
- Execute all cells automatically
- Generate updated visualizations
- Commit the results back to the repository
